# ETL Pipeline: PDF to PostgreSQL pgvector

이 노트북은 PDF 파일을 읽어서 마크다운으로 변환하고, 청크로 나눈 후 임베딩하여 PostgreSQL pgvector에 적재하는 전체 ETL 파이프라인을 구현합니다.

## Pipeline Overview
1. **Chapter-based Markdown Conversion**: 챕터 배열을 받아서 챕터별로 나눈 후 마크다운으로 변환
2. **Noise Removal**: 챕터 첫 페이지와 전체 페이지에서 노이즈 제거 (header/footer 패턴)
3. **Markdown Header Labeling**: 특정 패턴(Exercises, Key Terms 등)을 마크다운 헤더로 변환
4. **Header-based Chunking**: 마크다운 헤더를 기준으로 청크 분할
5. **Header-based Filtering**: 특정 헤더를 가진 청크 필터링
6. **Embedding with Clova**: Clova 임베딩 API를 사용한 벡터 생성 (RPM 고려)
7. **PostgreSQL pgvector Loading**: 임베딩 결과를 PostgreSQL pgvector에 적재

---
## Checkpoint 1: Chapter-based Markdown Conversion

PDF 파일에서 챕터 정보를 받아 각 챕터별로 페이지를 나누고 마크다운으로 변환합니다.

### 입력
- PDF 파일 경로
- 챕터별 시작 페이지 정보 (딕셔너리)

### 출력
- `chapter_markdowns`: 챕터별 마크다운 텍스트 (dict)

### 체크포인트 저장
- 변수: `checkpoint_1_chapter_markdowns`

### Step 1.1: 라이브러리 설치

필요한 라이브러리를 설치합니다.

In [ ]:
!pip install pymupdf4llm pymupdf -q

### Step 1.2: 라이브러리 임포트

필요한 라이브러리를 임포트합니다.

In [ ]:
import pymupdf4llm
import fitz  # PyMuPDF
import os

### Step 1.3: PDF 파일 및 챕터 정보 설정

처리할 PDF 파일 경로와 각 챕터의 시작 페이지 번호를 정의합니다.

In [ ]:
# PDF 파일 경로
pdf_path = "data/network.pdf"

# 챕터별 시작 페이지 번호 (PDF 페이지 번호)
chapter_start_pages = {
    "Chapter 1": 5,
    "Chapter 2": 47,
    "Chapter 3": 103,
    "Chapter 4": 179,
    "Chapter 5": 229,
    "Chapter 6": 287,
    "Chapter 7": 341,
    "Chapter 8": 373,
    "Chapter 9": 415,
}

# 챕터 정보 정렬 및 총 페이지 수 확인
sorted_chapters = sorted(chapter_start_pages.items(), key=lambda item: item[1])

doc = fitz.open(pdf_path)
total_pages = len(doc)
doc.close()

print(f"Target PDF: {pdf_path}")
print(f"Total Pages: {total_pages}")
print(f"Total Chapters: {len(sorted_chapters)}")
print(f"\nChapter Configuration:")
for chapter_name, start_page in sorted_chapters:
    print(f"  - {chapter_name}: Page {start_page}")

### Step 1.4: 챕터별 마크다운 변환 실행

각 챕터의 페이지 범위를 계산하고 pymupdf4llm을 사용하여 마크다운으로 변환합니다.

In [ ]:
chapter_markdowns = {}

print("=" * 60)
print("Starting Chapter-based Markdown Conversion")
print("=" * 60)

for i, (chapter_name, start_page) in enumerate(sorted_chapters):
    # 페이지 범위 계산 (0-indexed)
    start_idx = start_page - 1
    
    if i < len(sorted_chapters) - 1:
        # 다음 챕터 시작 전까지
        end_idx = sorted_chapters[i + 1][1] - 1
    else:
        # 마지막 챕터는 PDF 끝까지
        end_idx = total_pages
    
    # 페이지 범위 리스트 생성
    page_range = list(range(start_idx, end_idx))
    
    print(f"\n[{chapter_name}]")
    print(f"  Processing pages {start_idx + 1} to {end_idx} ({len(page_range)} pages)...")
    
    # pymupdf4llm을 사용하여 페이지별로 마크다운 변환
    chapter_md_text = pymupdf4llm.to_markdown(pdf_path, pages=page_range)
    
    # 챕터별 마크다운 저장
    chapter_markdowns[chapter_name] = chapter_md_text
    
    print(f"  ✓ Completed (Length: {len(chapter_md_text):,} characters)")

print("\n" + "=" * 60)
print(f"✓ All chapters converted successfully!")
print(f"  Total chapters: {len(chapter_markdowns)}")
print("=" * 60)

### Step 1.5: 체크포인트 저장 및 결과 확인

변환된 마크다운을 체크포인트로 저장하고 샘플 결과를 확인합니다.

In [ ]:
# 체크포인트 1 저장
checkpoint_1_chapter_markdowns = chapter_markdowns.copy()

print("✓ Checkpoint 1 saved successfully!")
print(f"  Variable: checkpoint_1_chapter_markdowns")
print(f"  Chapters: {list(checkpoint_1_chapter_markdowns.keys())}")
print(f"\n--- Sample: First 500 characters of Chapter 1 ---")
print(checkpoint_1_chapter_markdowns["Chapter 1"][:3000])
print("...")